# Transition Matrices: input data preparation

## 1. Aggregations by company and year

In [ ]:
import pandas as pd
import numpy as np
import os
from time import time

print('The script is running...')
t_start = time()

path_parent_dir = os.path.dirname(os.getcwd())
path_data = f'{path_parent_dir}\data'
path_data_web_scraper = f'{path_data}\web_scraper'
path_data_input = f'{path_data}\input_data'

df_fortune_rank = pd.read_csv(f'{path_data_web_scraper}/fortune_ranking_global_500.csv',sep=';')
df_fortune_rank['value_rec'] = np.where(df_fortune_rank['value']<=0, 0, df_fortune_rank['value'])
df_fortune_rank = df_fortune_rank.sort_values(by=['year','measure', 'value'], ascending=[True, True, False])                                  
df_fortune_rank = df_fortune_rank.reset_index(drop=True)  
df_fortune_rank['sum'] = df_fortune_rank.groupby(['year', 'measure'])['value_rec'].transform('sum').astype(int)
df_fortune_rank['cum_sum'] = df_fortune_rank.groupby(['year', 'measure'])['value_rec'].cumsum(axis=0).astype(int)
df_fortune_rank['cum_perc'] = np.where(df_fortune_rank['value_rec']<=0, 0, df_fortune_rank['cum_sum']/df_fortune_rank['sum'])
df_fortune_rank['rank'] = df_fortune_rank.groupby(['year', 'measure'])['value'].rank('dense', ascending=False)

## 2. Matrix class assignment

In [ ]:
conditions_class = [((df_fortune_rank['cum_perc']>0) & (df_fortune_rank['cum_perc']<=.1))
                    |
                    ((df_fortune_rank['rank']==1) & (df_fortune_rank['cum_perc']>0)),
                    (df_fortune_rank['cum_perc']>.1) & (df_fortune_rank['cum_perc']<=.2),
                    (df_fortune_rank['cum_perc']>.2) & (df_fortune_rank['cum_perc']<=.3),
                    (df_fortune_rank['cum_perc']>.3) & (df_fortune_rank['cum_perc']<=.4),
                    (df_fortune_rank['cum_perc']>.4) & (df_fortune_rank['cum_perc']<=.5),        
                    (df_fortune_rank['cum_perc']>.5) & (df_fortune_rank['cum_perc']<=.6),        
                    (df_fortune_rank['cum_perc']>.6) & (df_fortune_rank['cum_perc']<=.7),        
                    (df_fortune_rank['cum_perc']>.7) & (df_fortune_rank['cum_perc']<=.8),                     
                    (df_fortune_rank['cum_perc']>.8) & (df_fortune_rank['cum_perc']<=.9),                     
                    (df_fortune_rank['cum_perc']>.9),
                    (df_fortune_rank['cum_perc']<=0)]

actions_class = [']0% ; 10%]',']10% ; 20%]',']20% ; 30%]',']30% ; 40%]',']40% ; 50%]',
                 ']50% ; 60%]',']60% ; 70%]',']70% ; 80%]',']80% ; 90%]',']90% ; 100%]','<=0'] 

df_fortune_rank['matrix_class'] = np.select(conditions_class,actions_class,default='N/A')

## 3. Data pivoting and cleaning

In [ ]:
df_fortune_rank_tm = df_fortune_rank.iloc[:,np.r_[0:4,-2:0]].copy() 
df_fortune_rank_tm = df_fortune_rank_tm.pivot(index=['name','measure'], columns='year').reset_index()
df_fortune_rank_tm.columns = [f'{i}' if 'name' in i or 'measure' in i else f'{i}_{j}' for i, j in df_fortune_rank_tm.columns]
list_num_cols = df_fortune_rank_tm.select_dtypes(include=np.number).columns.tolist()
df_fortune_rank_tm[list_num_cols] = df_fortune_rank_tm[list_num_cols].fillna(0)
df_fortune_rank_tm.iloc[:,np.r_[-4:0]] = df_fortune_rank_tm.iloc[:,np.r_[-4:0]].fillna('na_class')
df_fortune_rank_tm.to_csv(f'{path_data_input}/transition_matrices_input.csv', header=True, index=False, encoding='utf-8-sig',sep=';')
print("...it has been successfully executed in %0.1fs." % (time() - t_start))